## Regex to label for KoBERT_02
- 01에서 진행했던 patients, medical staff 외 항목에 대한 labeling을 진행.
- 해당 노트까지 진행하면 노트는 regex에 잡히는 항목에 대해서는 모두 [... 'O' 'PER-B' 진행하였음 ....] 이런형태로 저장됨.
- regex에 잡히지 않는 평범한 문장은 원문 그대로 저장.

In [1]:
import os
import copy
import re

import pandas as pd

In [2]:
dat = pd.read_csv('./PHI_tagging/lb_test.csv', encoding='utf-8')
label = ["'PER-B'", "'PER-I'", "'DAT-B'", "'DAT-I'", "'ORG-B'", 
         "'ORG-I'", "'LOC-B'", "'LOC-I'", "'NUM-B'", "'NUM-I'", "'ETC-B'", "'ETC-I'"]

### Date
- date 폴더 내에 있는 모든 regex를 불러옴
- 패턴매칭되는 항목들에 대해 sub진행 

In [3]:
notes={}

for i in range(len(dat)):
    notes[dat['idx'][i]] = dat['label'][i]


In [5]:
ori = pd.read_csv('phis.csv', encoding='utf-8')

nori = {}

for i in range(len(dat)):
    nori[ori['note_id'][i]] = ori['note_text'][i]

save_origins = copy.deepcopy(nori)

In [6]:
patterns = []
date_regex_path = './filters/regex/dates/'
file_list = os.listdir(date_regex_path)

for file in file_list:
    if file.endswith('_transformed.txt'):
        with open(date_regex_path + file) as r:
            patterns.append(re.compile(r.readline()))

In [7]:
def convert_date(matchObj):
    match = matchObj.group()
    convertString = "'DAT-B'" + " 'DAT-I'" * (len(match.split())-1)
    return convertString

In [8]:
transformed = pd.DataFrame(columns = ['idx', 'origin', 'label'])

i=0
for k in notes.keys():
    
    convNote = notes[k]
    
    for p in patterns:
        convNote = re.sub(p, convert_date, convNote)
    
    newNotes = [] #label정보를 담고 있음
    for words in convNote.split():
        if words in label:
            newNotes.append(words)
        else:
            newNotes.append("'O'")    
#     for i in range(len(newNotes)-1):
#         print(notes[notes].split()[i] + '\t' + newNotes[i])
    transformed.loc[i] = [k, notes[k], convNote]
    i += 1

In [9]:
transformed[:3]

,idx,origin,label
0,301629252,[Conclusion]\n\r1. Thyroid Parenchyma\r\n - n...,[Conclusion]\n\r1. Thyroid Parenchyma\r\n - n...
1,302222272,"[Conclusion]\n\rImproved probable pneumonia, R...","[Conclusion]\n\rImproved probable pneumonia, R..."
2,308580103,[Conclusion]\n\rNo active lesion in the lung.\...,[Conclusion]\n\rNo active lesion in the lung.\...


---

### Hospitals

In [10]:
dat = transformed

for i in range(len(dat)):
    notes[dat['idx'][i]] = dat['label'][i]

In [11]:
patterns = []
date_regex_path = './filters/regex/organizations/'
file_list = os.listdir(date_regex_path)

for file in file_list:
    if file.endswith('_transformed.txt'):
        with open(date_regex_path + file) as r:
            patterns.append(re.compile(r.readline()))

In [12]:
def convert_orgs(matchObj):
    match = matchObj.group()
    convertString = "'ORG-B'" + " 'ORG-I'" * (len(match.split())-1)
    return convertString

In [13]:
transformed2 = pd.DataFrame(columns = ['idx', 'origin', 'label'])

In [14]:
i=0
for k in notes.keys():
    
    convNote = notes[k]
    
    for p in patterns:
        convNote = re.sub(p, convert_orgs, convNote)
    
    newNotes = [] #label정보를 담고 있음
    for words in convNote.split():
        if words in label:
            newNotes.append(words)
        else:
            newNotes.append("'O'")    
#     for i in range(len(newNotes)-1):
#         print(notes[notes].split()[i] + '\t' + newNotes[i])
    transformed2.loc[i] = [k, notes[k], convNote]
    i += 1

### Region

In [15]:
dat = transformed2

for i in range(len(dat)):
    notes[dat['idx'][i]] = dat['label'][i]

In [16]:
regex = './filters/regex/region/district_kor_transformed.txt'

with open(regex) as r:
    patterns = re.compile(r.readline())
    #re.compile(r.readline())
    
def convert_region(matchObj):
    match = matchObj.group()
    convertString = "'LOC-B'"
    return convertString

In [ ]:
transformed3 = pd.DataFrame(columns = ['idx', 'origin', 'label'])

i=0
for k in notes.keys():
    convNote = re.sub(patterns, convert_region, notes[k])
    newNotes = [] #label정보를 담고 있음
    for words in convNote.split():
        if words in label:
            newNotes.append(words)
        else:
            newNotes.append("'O'")    
#     for i in range(len(newNotes)-1):
#         print(notes[notes].split()[i] + '\t' + newNotes[i])
    transformed3.loc[i] = [k, notes[k], convNote]
    i += 1

In [ ]:
transformed3[:3]

### Phone

In [ ]:
dat = transformed3

for i in range(len(dat)):
    notes[dat['idx'][i]] = dat['label'][i]

In [ ]:
regex = './filters/regex/phone/nums.txt'

with open(regex) as r:
    patterns = re.compile(r.readline())
    #re.compile(r.readline())
    
def convert_nums(matchObj):
    match = matchObj.group()
    convertString = "'NUM-B'" + " 'NUM-I'" * (len(match.split())-1)
    return convertString

In [ ]:
transformed4 = pd.DataFrame(columns = ['idx', 'origin', 'label'])

i=0
for k in notes.keys():
    convNote = re.sub(patterns, convert_nums, notes[k])
    newNotes = [] #label정보를 담고 있음
    for words in convNote.split():
        if words in label:
            newNotes.append(words)
        else:
            newNotes.append("'O'")    
#     for i in range(len(newNotes)-1):
#         print(notes[notes].split()[i] + '\t' + newNotes[i])
    transformed4.loc[i] = [k, notes[k], convNote]
    i += 1

### etc

In [ ]:
dat = transformed4

for i in range(len(dat)):
    notes[dat['idx'][i]] = dat['label'][i]

In [ ]:
patterns = []
date_regex_path = './filters/regex/etc/'
file_list = os.listdir(date_regex_path)

for file in file_list:
    if file.endswith('_transformed.txt'):
        with open(date_regex_path + file) as r:
            patterns.append(re.compile(r.readline()))

In [ ]:
def convert_etc(matchObj):
    match = matchObj.group()
    convertString = "'ETC-B'" + " 'ETC-I'" * (len(match.split())-1)
    return convertString

In [ ]:
transformed5 = pd.DataFrame(columns = ['idx', 'origin', 'label'])

i=0
for k in notes.keys():
    
    convNote = notes[k]
    
    for p in patterns:
        convNote = re.sub(p, convert_etc, convNote)
    
    newNotes = [] #label정보를 담고 있음
    for words in convNote.split():
        if words in label:
            newNotes.append(words)
        else:
            newNotes.append("'O'")    
#     for i in range(len(newNotes)-1):
#         print(notes[notes].split()[i] + '\t' + newNotes[i])
    transformed5.loc[i] = [k, save_origins[k], convNote]
    i += 1

In [ ]:
transformed5.to_csv('./PHI_tagging/lb_all_test.csv', index=False)